## Data

In [1]:
import torch
import torch.nn as nn
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "i like a dog",
    "books are expensive"    
]
data_size = len(corpus)

# 0: noun/pronoun - 1: verb - others - 2
labels = [[0, 1, 2, 0],
          [0, 1, 2]]

# Define the max vocabulary size and sequence length
vocab_size = 9
sequence_length = 4

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'dog': 5,
 'books': 4,
 'like': 8,
 'i': 7,
 'are': 3,
 'a': 2,
 'expensive': 6,
 '<pad>': 1,
 '<unk>': 0}

In [3]:
# Tokenize and numericalize your samples
def vectorize(text, vocab, sequence_length, sequence_label):
    tokens = tokenizer(text)
    
    token_ids = [vocab[token] for token in tokens]    
    token_ids = token_ids + [vocab["<pad>"]] * (sequence_length - len(tokens))
    sequence_label = sequence_label + [3] * (sequence_length - len(tokens))
    
    return torch.tensor(token_ids, dtype=torch.long), torch.tensor(sequence_label, dtype=torch.long)

# Vectorize the samples
sentence_vecs = []
label_vecs = []
for sentence, labels in zip(corpus, labels):
    sentence_vec, labels_vec = vectorize(sentence, vocab, sequence_length, labels)
    sentence_vecs.append(sentence_vec)
    label_vecs.append(labels_vec)

In [4]:
for v in sentence_vecs:
    print(v)

tensor([7, 8, 2, 5])
tensor([4, 3, 6, 1])


In [5]:
for v in label_vecs:
    print(v)

tensor([0, 1, 2, 0])
tensor([0, 1, 2, 3])


## Model

In [6]:
class POS_Model(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 4)
        self.recurrent = nn.RNN(4, 3, 
                                batch_first=True)
        self.fc = nn.Linear(3, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.recurrent(x)
        x = self.fc(x)
        return x.permute(0, 2, 1)

num_classes = 4
model = POS_Model(vocab_size, num_classes)

# test
input = torch.tensor([[7, 8, 2, 5]], dtype=torch.long)
output = model(input)
print(output.shape)

torch.Size([1, 4, 4])


## Train with full data

In [7]:
criterion = nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)

In [8]:
input_data = torch.tensor( [[7, 8, 2, 5],
                            [4, 3, 6, 1]], dtype=torch.long)
label_data = torch.tensor([[0, 1, 2, 0],
                           [0, 1, 2, 3]], dtype=torch.long)

for _ in range(30):
    optimizer.zero_grad()
    outputs = model(input_data)
    loss = criterion(outputs, label_data)
    print(loss.item())
    loss.backward()
    optimizer.step()

1.4923657178878784
1.1042066812515259
0.8223642110824585
0.5547635555267334
0.3815281093120575
0.2592349946498871
0.1677534431219101
0.10552334785461426
0.0747343972325325
0.057824425399303436
0.04507506638765335
0.03518969565629959
0.027546923607587814
0.021697646006941795
0.01724390499293804
0.013850478455424309
0.011252852156758308
0.009250220842659473
0.007693153340369463
0.00647133169695735
0.005503506865352392
0.004729758016765118
0.004105450119823217
0.003597244620323181
0.0031801010482013226
0.0028349668718874454
0.0025473323184996843
0.0023060201201587915
0.0021022127475589514
0.001929137040860951


In [9]:
outputs = model(input_data)
o_softmax = torch.softmax(outputs, axis=1)

print(o_softmax[0, :, 0].detach().numpy())
print(o_softmax[0, :, 1].detach().numpy())
print(o_softmax[0, :, 2].detach().numpy())
print(o_softmax[0, :, 3].detach().numpy())

print()
print(o_softmax[1, :, 0].detach().numpy())
print(o_softmax[1, :, 1].detach().numpy())
print(o_softmax[1, :, 2].detach().numpy())
print(o_softmax[1, :, 3].detach().numpy())

[9.9809068e-01 8.0252049e-04 1.1050679e-03 1.6739086e-06]
[2.3262764e-03 9.9742329e-01 2.3816977e-04 1.2234963e-05]
[3.0269544e-05 1.0362170e-03 9.9881279e-01 1.2070947e-04]
[9.9809164e-01 8.0280745e-04 1.1039404e-03 1.6731840e-06]

[9.9809164e-01 8.0272747e-04 1.1039563e-03 1.6730722e-06]
[2.3265595e-03 9.9742293e-01 2.3833601e-04 1.2219893e-05]
[6.2340972e-05 2.5850115e-04 9.9961120e-01 6.8069523e-05]
[8.1286979e-01 1.8086505e-01 6.2229233e-03 4.2267031e-05]


In [10]:
# [[0, 1, 2, 0],
#  [0, 1, 2, *]]

o_softmax.argmax(axis=1)

tensor([[0, 1, 2, 0],
        [0, 1, 2, 0]])